### le rose sono rosse / le viole sono blu / prendetemi per pazzo, sì / pazzo per Gesù

In [ ]:
import grembi as fnz
import pickle
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np
import csv

In [ ]:
NE = 'north_east'
SW = 'south_west'
view = {
'chinatown': {SW:(45.478530132389416,9.167947775395564),NE:(45.486329944279085,9.181335323890014),},
#'legit_milan': {SW:(45.43654061758556,9.1255932966908),NE:(45.52335073625751,9.25182332813564),},
'vittoria_dateo': {SW:(45.46218301808607, 9.206638223420574),NE:(45.468007095952004, 9.217281117503491),},
'navigli_romana': {SW:(45.44416732498216, 9.158639723868536),NE:(45.452934043919186, 9.207787100677395),},
}

In [ ]:
n_reviews = 400
meters = 150
port = 'navigli_romana'

# calls payout
len(fnz.cute_types)*len(fnz.get_radars(meters, **view[port]))

##### API places

In [ ]:
#pickle.dump(fnz.sample_viewport(meters, view[port]), open('cetriolini/'+port+"_places_db.pkl", "wb"))

In [ ]:
places_db = pickle.load(open('cetriolini/'+port+"_places_db.pkl", "rb")) # list of dict-place

##### purify dataset

In [ ]:
#pickle.dump(fnz.purify_data(places_db), open('cetriolini/'+port+"_data_low.pkl", "wb"))

In [ ]:
data_low = pickle.load(open('cetriolini/'+port+"_data_low.pkl", "rb"))

##### scraping reviews

In [ ]:
#pickle.dump(fnz.scrape_reviews([i['place_id'] for i in data_low], n_reviews), open('cetriolini/'+port+"_reviews_db.pkl", "wb"))

In [ ]:
reviews_db = pickle.load(open('cetriolini/'+port+"_reviews_db.pkl", "rb"))  # dict of list of htmls <- to BeautifulSoup(htmls)

##### upgrade data

In [ ]:
#pickle.dump(fnz.enrich_data(data_low, reviews_db), open('cetriolini/'+port+"_data_high.pkl", "wb"))

In [ ]:
data_high = pickle.load(open('cetriolini/'+port+"_data_high.pkl", "rb"))  # beautiful json

##### merging

In [ ]:
data = []
for i in view:
    data += pickle.load(open('cetriolini/'+i+"_data_high.pkl", "rb"))

In [ ]:
data = [i for i in data if i['user_ratings_total'] > 5 ] # 6 or more

In [ ]:
data = [i for i in data if i['price_level']] # assigned price

In [ ]:
data = [i for i in data if [j for j in i['rewiews'] if j['days_ago'] < 365]]

In [ ]:
reg = []
x_head = ['price_level','rating','user_ratings_total',]
y_head = ['is_elite','rank','stars','days_ago',]
for p in data:
    x = {i: p[i] for i in x_head}
    for r in p['rewiews']:
        y = {i: r[i] for i in y_head}
        reg.append({**x, **y})

In [ ]:
f = open("data.csv", "w")
writer = csv.DictWriter(f, fieldnames= x_head + y_head)
writer.writeheader()
writer.writerows(reg)
f.close()

##### stats

In [ ]:
len(reg)

In [ ]:
a = [i['user_ratings_total'] for i in data if i]

In [ ]:
fig = plt.figure()#figsize=sq_leaf)
ax = fig.gca()
eps = 1
y, x = np.histogram(a, np.arange(0, max(a)+2*eps, eps))
ax.plot(x[:-1], y, marker='.',ms=3.0, linewidth=0.0, linestyle='-', color='purple')
ax.set_xlim(1, 30)
#ax.axes.get_yaxis().set_visible(False)
#ax.axvline(x=numpy.mean(a), color='blue', linewidth=1.2)
#ax.axvline(x=numpy.median(a), color='red', linewidth=1.2)
#ax.text(16, 300000,'$\mu$')
#ax.text(6, 30000,'$0.5q$')

In [ ]:
plt.boxplot(np.log10(a))